In [1]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelBinarizer

from sklearn.preprocessing import PolynomialFeatures
from mlxtend.classifier import StackingClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

from My_Transformers import *
from Word2Vec_Transformer import WVEmbeddingTransformer, Text_cleaner_transformer, EMB_VOCAB_SET, WV_EMBEDDINGS

import warnings
warnings.filterwarnings("ignore")

importing Jupyter notebook from My_Transformers.ipynb
importing Jupyter notebook from Word2Vec_Transformer.ipynb


2020-09-15 10:05:57,548 INFO adding document #0 to Dictionary(0 unique tokens: [])
2020-09-15 10:05:57,549 INFO built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
def pickleSave (obj, file):
    
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

In [3]:
def unpickle (file):
    return pickle.load (open (file, 'rb'))

In [4]:
def preProcess (rawfile, newFileName):
    
    df = pd.read_csv (rawfile)
    Xstr = df['text']
    text_cleaner = Text_cleaner_transformer (isCorrection=True, embedding=WV_EMBEDDINGS)
    Xstr = text_cleaner.transform (Xstr)
    df['text'] = Xstr
    df.to_csv (newFileName)
    return

In [5]:
train_file = "Data/train.csv"
preprocessed_train_file = "Data/preprocessed_train.csv"
# preProcess (train_file, preprocessed_train_file)
df = pd.read_csv (preprocessed_train_file)
# df = df[0:20]
df['keyword'] = df.keyword.astype(str)
keywords_train = list (df['keyword'])
LB = LabelBinarizer () 
LB.fit (keywords_train)
LB_loc = LabelBinarizer () 
df['location'] = LB_loc.fit_transform (list (pd.isnull (df['location'])))

In [6]:
Xstr = df["text"]
y = df["target"]

In [7]:
Xstr_transformer = FeatureUnion ([
    
    ('wv_embed', WVEmbeddingTransformer ('D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin', vocabSet=EMB_VOCAB_SET, isCorrection=True)),
    ('tfidf_latent', Pipeline ([
        
        ('text_clean',       Text_cleaner_transformer ()),
        ('tfidf_vectorizer', TfidfVectorizer (analyzer='char', ngram_range=(3,7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
        ('interactions',     PolynomialFeatures (2, interaction_only=True, include_bias=False)),
        ('tfidf_latent',     FeatureUnion ([
            
            ('latent', Pipeline ([
                
                ('feature_union',   FeatureUnion ([
                    ("svd_nmf_select", SVD_NMF_FeatureTransformer ()),
                    ('lda',            LDA_FeatureTransformer ()),
                    ('kpca',           KPCASparse_FeatureTransformer (kernel=['rbf'])),
                    # ('ica',          ICASparse_FeatureTransformer ()),   # does not works with sparse-matrices
                ])),
                ('k_best_feat_select_chi2', SelectKBest_feature_selector (score_func=[mutual_info_classif])), # should not be used with -ve vals
            ])),
            ('tfidf', Pipeline ([
                
                ('sparse_svm_feat_select',  SparseSVM_feature_selector ()),
            ]))
        ]))
    ]))
])

2020-09-15 10:05:59,366 INFO loading projection weights from D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin
2020-09-15 10:06:20,535 INFO loaded (3000000, 300) matrix from D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin


In [ ]:
X = scipy.sparse.hstack ((Xstr_transformer.fit_transform (Xstr, y), LB.transform (keywords_train), LB_loc.transform (list (pd.isnull (df['location'])))))
X = X.tocsr()
rf_importance_feature_selector = CLF_importance_feature_selector()
X = rf_importance_feature_selector.fit_transform (X, y)
# pickleSave (X, 'X.bin')
# X = unpickle ('X.bin')
X.shape

In [ ]:
pickleSave (Xstr_transformer, 'Xstr_transformer.bin')
# Xstr_transformer = unpickle ('Xstr_transformer.bin')
pickleSave (rf_importance_feature_selector, 'rf_importance_feature_selector.bin')

In [ ]:
# try these classifiers: best = logisticReg cv=73.4%
params_logit = { 'name': 'Logit', 'params': {'penalty': 'l2', 'class_weight': 'balanced'}, \
                                               'param_grid': {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 1], 'fit_intercept': [True, False]} }
params_rf    = { 'name': 'RF',    'params': {'class_weight': 'balanced'} } 
params_lsvc  = { 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, \
                                                                  'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1]} }
params_xgb   = { 'name': 'Xgb',   'param_grid':{'max_depth': [3, 5], 'learning_rate':[1.5, 3]} }
params_catb  = { 'name': 'Catb'}
params_bnnb  = { 'name': 'BernNB'}
params_nerc  = { 'name': 'NearCent'}
params_ridc  = { 'name': 'RidgClf'}
params_psag  = { 'name': 'PssAggClf'}


params = [params_logit, params_rf, params_lsvc, params_xgb, params_catb, params_bnnb, params_nerc, params_ridc, params_psag]
bclfs  = []
for param in params:
    
    bclf = Best_clf_cv_transformer (param)
    bclf.fit (X, y) 
    bclfs.append ((bclf, bclf.get_cv_score()))
# Choose the bclf with the best cv_score:
bclf = max (bclfs, key = lambda i : i[1])[0]
pickleSave (bclfs, 'bclfs.bin')
# bclfs = unpickle ('bclfs.bin')
bclf

In [ ]:
test_file = "Data/test.csv"
preprocessed_test_file = "Data/preprocessed_test.csv"
# preProcess (test_file, preprocessed_test_file)

In [ ]:
testDF = pd.read_csv (preprocessed_test_file)
# testDF = testDF[0:10]
testDF['keyword'] = testDF.keyword.astype(str)
testDF.shape

In [ ]:
X_test_str = testDF["text"]
X_test = scipy.sparse.hstack ((Xstr_transformer.transform (X_test_str), LB.transform (list (testDF['keyword'])), LB_loc.transform (list (pd.isnull (testDF['location'])))))
X_test = X_test.tocsr()
X_test = rf_importance_feature_selector.transform (X_test)
# pickleSave (X_test, 'X_test.bin')
X_test.shape

In [ ]:
for bclf in bclfs:
    
    bclf = bclf[0]
    predictions = bclf.predict (X_test)
    testDF['target'] = predictions
    submitDF = testDF[['id', 'target']]
    submitDF.to_csv ('submission_v4_'+bclf.name+'.csv', index=False)